
frequency calendar index?

ROUTE = /frequency


In [ ]:
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Event Frequency Visualizer</title>
  <style>
    body { font-family: sans-serif; margin: 20px; }
    .controls { margin-bottom: 20px; }
    label { display: block; margin-top: 10px; }
    select, input, button { padding: 6px; margin-top: 4px; }
    .chart { margin-top: 40px; }
    svg { width: 100%; height: auto; }
  </style>
</head>
<body>

  <h1>Event Frequency Visualizer + Swimlane Viewer</h1>

  <div class="controls">
    <label for="calendar">Select Calendar:</label>
    <select id="calendar">
      <option value="Work">Work</option>
      <option value="Personal">Personal</option>
      <option value="Emotions">Emotions</option>
    </select>

    <label for="keywords">Enter Keywords (comma-separated):</label>
    <input type="text" id="keywords" placeholder="e.g. gym, meeting, focus" />

    <label for="viz">Select Visualization:</label>
    <select id="viz">
      <option value="pie">Pie Chart</option>
      <option value="heatmap">Heatmap</option>
      <option value="line">Line Chart</option>
      <option value="swimlane">Swimlane</option>
    </select>

    <button onclick="generate()">Generate</button>
  </div>

  <div id="chart" class="chart"></div>

  <script src="https://d3js.org/d3.v7.min.js"></script>
  <script src="/frequency-client.js"></script>
/body>
</html>

# 📊 Event Frequency Visualizer + Swimlane Viewer

This interactive UI lets you:
- Select a Google Calendar
- Input keywords (e.g., "focus", "gym", "meeting")
- Choose a visualization:
  - Pie Chart (keyword frequency)
  - Heatmap (daily density)
  - Line Chart (keyword trend)
  - Swimlane (timeline by keyword)

🔧 It simulates event data for now — plug in your backend logic (e.g., Google Calendar API) to retrieve real events.

---

Try different visualizations and keywords to explore your time use trends!


calendar frequency timeline?

ROUTE = /frequency-client.js


In [ ]:
async function listEvents(calendarId, keyword) {
  const now = new Date();
  const events = [];
  for (let i = 0; i < 30; i++) {
    let date = new Date(now);
    date.setDate(now.getDate() - i);
    events.push({
      id: `${keyword}-${i}`,
      start: new Date(date.getFullYear(), date.getMonth(), date.getDate(), Math.random() * 24),
      end: new Date(date.getFullYear(), date.getMonth(), date.getDate(), Math.random() * 24 + 1),
      keyword
    });
  }
  return events;
}

async function generate() {
  const calendarId = document.getElementById('calendar').value;
  const keywords = document.getElementById('keywords').value.split(',').map(k => k.trim());
  const viz = document.getElementById('viz').value;
  const allEvents = (await Promise.all(keywords.map(k => listEvents(calendarId, k)))).flat();
  renderChart(viz, allEvents, keywords);
}

function renderChart(type, data, keywords) {
  document.getElementById('chart').innerHTML = '';
  const svg = d3.select('#chart').append('svg').attr('width', 800).attr('height', 400);

  if (type === 'pie') {
    const counts = d3.rollup(data, v => v.length, d => d.keyword);
    const pie = d3.pie().value(d => d[1])(Array.from(counts.entries()));
    const arc = d3.arc().innerRadius(0).outerRadius(150);
    const g = svg.append('g').attr('transform', 'translate(300,200)');
    g.selectAll('path')
      .data(pie)
      .enter()
      .append('path')
      .attr('d', arc)
      .attr('fill', (d, i) => d3.schemeCategory10[i]);
  }

  if (type === 'heatmap') {
    const days = d3.timeDays(d3.min(data, d => d.start), d3.max(data, d => d.start));
    const counts = d3.rollup(data, v => v.length, d => d.start.toDateString());
    svg.selectAll('rect')
      .data(days)
      .enter()
      .append('rect')
      .attr('x', (d, i) => i * 20)
      .attr('y', 50)
      .attr('width', 18)
      .attr('height', 100)
      .attr('fill', d => d3.interpolateReds((counts.get(d.toDateString()) || 0) / 10));
  }

  if (type === 'line') {
    const counts = d3.rollups(data, v => v.length, d => d.start.toDateString());
    const x = d3.scalePoint().domain(counts.map(d => d[0])).range([50, 750]);
    const y = d3.scaleLinear().domain([0, d3.max(counts, d => d[1])]).range([350, 50]);
    const line = d3.line().x(d => x(d[0])).y(d => y(d[1]));
    svg.append('path')
      .datum(counts)
      .attr('fill', 'none')
      .attr('stroke', 'steelblue')
      .attr('stroke-width', 2)
      .attr('d', line);
  }

  if (type === 'swimlane') {
    const laneHeight = 40;
    keywords.forEach((kw, i) => {
      const lane = svg.append('g').attr('transform', `translate(0, ${i * laneHeight})`);
      lane.append('text').text(kw).attr('x', 0).attr('y', 30);
      data.filter(d => d.keyword === kw).forEach(ev => {
        lane.append('rect')
          .attr('x', (ev.start.getHours() / 24) * 700 + 50)
          .attr('y', 10)
          .attr('width', ((ev.end - ev.start) / (1000 * 60 * 60)) * (700 / 24))
          .attr('height', 20)
          .attr('fill', 'orange')
          .append('title').text(`${ev.keyword}: ${ev.start.toLocaleString()}`);
      });
    });
  }
}
